In [1]:
import os
import pandas as pd

DATASET_PATH = r"C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET"

csv_files = [
    os.path.join(DATASET_PATH, f)
    for f in os.listdir(DATASET_PATH)
    if f.endswith(".csv")
]

df = pd.DataFrame()

for file in csv_files:
    print("Loading:", file)

    for chunk in pd.read_csv(file, chunksize=200000):
        df = pd.concat([df, chunk], ignore_index=True)

print("Final shape:", df.shape)


Loading: C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Loading: C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Loading: C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET\Friday-WorkingHours-Morning.pcap_ISCX.csv
Loading: C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET\Monday-WorkingHours.pcap_ISCX.csv
Loading: C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET\SMOTE_resampled_dataset.csv
Loading: C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Loading: C:\Users\sudee\OneDrive\Documents\GitHub\MINOR_2-NETWORK-ANAMOLY-DETECTION\DATASET\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Loading: C:\Users\sudee\OneDrive\Do

In [2]:
df.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4330743 entries, 0 to 4330742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [4]:
df.columns = df.columns.str.strip()


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4330743 entries, 0 to 4330742
Data columns (total 79 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             int64  
 1   Flow Duration                int64  
 2   Total Fwd Packets            int64  
 3   Total Backward Packets       int64  
 4   Total Length of Fwd Packets  int64  
 5   Total Length of Bwd Packets  int64  
 6   Fwd Packet Length Max        int64  
 7   Fwd Packet Length Min        int64  
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        int64  
 11  Bwd Packet Length Min        int64  
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 int64  
 19  

In [6]:
df=df.drop(columns=['Fwd Avg Bytes/Bulk',
    'Fwd Avg Packets/Bulk',
    'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk',
    'Bwd Avg Packets/Bulk',
    'Bwd Avg Bulk Rate'])

In [7]:
def mergeanomalies(label):
    if label in [
        "DDoS",
        "DoS Hulk",
        "DoS GoldenEye",
        "DoS Slowhttptest",
        "DoS slowloris"
    ]:
        return "1"
    elif label=="PortScan":
        return "2"

    elif label in [
        "FTP-Patator",
        "SSH-Patator",
        "Web Attack – Brute Force"
    ]:
        return "3"

    elif label=="BENIGN":
        return "0"
    else:
        return "4"


In [9]:
df["Label"] = df["Label"].apply(mergeanomalies)


In [11]:
print(df["Label"].value_counts())


Label
0    2273097
4    1504193
1     380688
2     158930
3      13835
Name: count, dtype: int64


In [13]:
df.to_csv("processed_dataset.csv", index=False)